# JavaScriptでコンテンツが生成されるサイトのスクレイピング【Selenium+PhantomJS】

## 流れ

1. SeleniumとPhantomJSでJavaScriptサイトからHTMLをスクレイピング
2. 抽出したHTMLをBeautifulSoup4でパースし、必要な情報を抽出
3. MongoDBにデータを保存
4. csv形式でデータを保存
5. RSS形式でデータを保存

## SeleniumとPhantomJSでJavaScriptサイトからHTMLをスクレイピング

- pip install selenium
- PhantomJSのインストール

※`AttributeError: 'NoneType' object has no attribute 'attrs'`が出る場合、`sleep_time`の数を大きくして再実行してみてください。

In [ ]:
import sys
import time
from selenium import webdriver # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup


from logging import getLogger, StreamHandler, DEBUG
logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.setLevel(DEBUG)
logger.addHandler(handler)
logger.propagate = False

# urlを指定
url = 'https://note.mu/'

# 読み込む投稿の多さを指定(amount * 10 投稿程度)
amount = 5

# 読み込み時に待機する時間の指定
sleep_time = 7


def main():
    """
    メイン処理
    """
    
    # PhantomJS本体のパスを指定
    pjs_path = r"C:\phantomjs-2.1.1\bin\phantomjs.exe"
    driver = webdriver.PhantomJS(executable_path=pjs_path)
    
    # ページの読み込み
    navigate(driver, url, amount=amount, sleep_time=sleep_time)
    
    # データの抽出
    posts = scrape_posts(driver, url)
        
    return posts


def navigate(driver, url, amount=1, sleep_time = 5):
    """
    目的のページに遷移する
    amount >= 1
    """
    
    logger.debug('Navigating...')
    driver.get(url)
    assert 'note' in driver.title

    # 指定した回数分、ページ下部までスクロールしてコンテンツの生成を待つ
    for i in range(1, amount+1):
        driver.execute_script('scroll(0, document.body.scrollHeight)')
        logger.debug('Waiting for contents to be loaded...({0} times)'.format(i))
        time.sleep(sleep_time)
    

def scrape_posts(driver, url):
    """
    投稿のURL、タイトル、概要のdictをリスト形式で取得
    """
    
    posts = []
    
    # Seleniumで取得したHTMLをBeautifulSoup4に読み込む
    html = driver.page_source
    bsObj = BeautifulSoup(html,"html.parser")

    for post_html in bsObj.findAll("div",{"class":"c-card__body"}):
        # 記事のURLを取得
        content_url = post_html.find("h3").find('a').attrs['href']
        content_full_url = url + content_url[1:]

        # 記事タイトルを取得
        title = post_html.find("h3").find('a').find('span').get_text()
        title = title.replace('\n', '') # 改行を削除

        # 記事の概要を取得
        try:
            description = post_html.find("p", {'class':'p-cardItem__description'}).get_text()
            description = description.replace('\n', '') # 改行を削除
        except AttributeError as e:
            description = '-no description-'
            logger.debug("「{0}」 has no description: {1}".format(title, e))

        posts.append({
            'url': content_full_url,
            'title': title,
            'description': description,
            })
    
    return posts

# if __name__ == '__main__':
#     main()    

In [ ]:
posts = main()

## MongoDBにデータを保存

mongoDBを起動しておく

`mongod --dbpath "${データベース用のディレクトリへのパス}"`

In [ ]:
from pymongo import MongoClient, DESCENDING # pip install pymongo

mongo_client = MongoClient('localhost', 27017) # MongoDBと接続
db = mongo_client.note
collection = db.recomend # noteデータベース -> recomendコレクション
collection.delete_many({}) # 既存の全てのドキュメントを削除しておく

In [ ]:
def save_to_mongodb(collection, items):
    """
    MongoDBにアイテムのリストを保存
    """

    result = collection.insert_many(items) # コレクションに挿入
    logger.debug('Inserted {0} documents'.format(len(result.inserted_ids)))

In [ ]:
save_to_mongodb(collection, posts)

## csv形式でデータを保存

In [ ]:
import csv

def save_as_csv(posts, csv_name):
    # 列名（1行目）を作成
    ## [タイトル、URL、概要]
    col_name = ['title', 'url', 'description']

    with open(csv_name, 'w', newline='', encoding='utf-8') as output_csv:
        csv_writer = csv.writer(output_csv)
        csv_writer.writerow(col_name) # 列名を記入

        # データを整形しつつcsvに書き込んでいく
        for post in posts:
            row_items = [post['title'], post['url'], post['description']]
            csv_writer.writerow(row_items)

In [ ]:
# =====
# MongoDBを経由する場合
# from pymongo import MongoClient, DESCENDING # pip install pymongo

# mongo_client = MongoClient('localhost', 27017) # MongoDBと接続
# db = mongo_client.note
# collection = db.recomend # noteデータベース -> recomendコレクション
# posts = collection.find()

# MongoDBを使用しない場合
posts = posts
# =====

csv_name = 'note_list.csv'

save_as_csv(posts, csv_name)

## RSSフィードの生成

In [ ]:
import feedgenerator # pip install feedgenerator

def save_as_feed(f, posts):
    """
    コンテンツリストをRSSフィードとして保存
    """
    
    feed = feedgenerator.Rss201rev2Feed(
        title='おすすめノート',
        link='https://note.mu/',
        description='おすすめノート')
    
    for post in posts:
        feed.add_item(title=post['title'], link=post['url'],
                     description=post['description'], unique_id=post['url'])
        
    feed.write(f, 'utf-8')

In [ ]:
# =====
# MongoDBを経由する場合
# from pymongo import MongoClient, DESCENDING # pip install pymongo

# mongo_client = MongoClient('localhost', 27017) # MongoDBと接続
# db = mongo_client.note
# collection = db.recomend # noteデータベース -> recomendコレクション
# posts = collection.find()

# MongoDBを使用しない場合
posts = posts
# =====

with open('note_recommend.rss', 'w', encoding='utf-8') as f:
    save_as_feed(f, posts)